<a href="https://colab.research.google.com/github/thashmadech/is417-project/blob/main/Sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import dask
import re, string, unicodedata
from gensim.parsing.preprocessing import remove_stopwords
import nltk
from nltk import ngrams, word_tokenize, sent_tokenize, FreqDist
from nltk.tokenize import TweetTokenizer
from tqdm import tqdm
from textblob import TextBlob


# plotting
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt



In [ ]:
df_tweets = pd.read_csv('/content/drive/MyDrive/is417-project/Bitcoin_tweets.csv',engine='python',encoding='utf-8', error_bad_lines=False, dtype='str')


In [ ]:
df_tweets.sample(20)

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
314529,Jorijn,The Netherlands,Freelance Web Developer. Wants to automate eve...,2008-03-25 20:15:57,773.0,693.0,19.0,False,2021-07-03 13:56:32,I really like automating stuff. #Bitcoin DCA's...,['Bitcoin'],Twitter Web App,False
537159,eth,NaN,drink more water,2021-04-26 03:52:17,22.0,198.0,1895.0,False,2021-07-19 03:38:20,This could get ugly real fast #bitcoin #bearish,"['bitcoin', 'bearish']",Twitter for iPhone,False
1128497,#Help Me Pay My Bills,Global,https://t.co/vBGrS2H9me\n\nPutting the Lightni...,2015-05-02 16:54:25,77118.0,55353.0,127576.0,False,2021-08-25 13:36:54,https://t.co/36POQXZydV What traders should an...,"['xrp', 'Bitcoin', 'bounty']",Twibble.io,False
303792,Blue Collar Bitcoin Podcast,NaN,Firefighters explore the most important moneta...,2021-05-18 00:53:56,80.0,187.0,21.0,False,2021-07-03 23:15:46,@AriZonanHODL Buy 2 more #Bitcoin and put thei...,['Bitcoin'],Twitter for Android,False
81868,BitcoinTracker,NaN,Tracking Bitcoin Prices and Moving Averages wi...,2021-02-17 21:05:57,313.0,7.0,13.0,False,2021-04-23 06:33:36,"$BTCUSD\nCurrent Bitcoin Price: $48,968\nDay H...","['Bitcoin', 'RealMoney', 'BTC', 'Cypto']",BitCoinTrackerBTC,False
886567,Stacking Sats 🇺🇸,"Las Vegas, NV",#Bitcoin,2013-08-22 18:56:24,2762.0,720.0,39249.0,False,2021-08-20 03:27:59,Where does it stop? What if they are White? Re...,['bitcoin'],Twitter for iPhone,False
1190319,BitcoinPress,NaN,Bitcoin and Blockchain news media platform.,2013-10-27 19:11:22,522.0,72.0,161.0,False,2021-08-26 01:46:32,"#Litecoin #BTC #DOGE Bitcoin, ether, dogecoin ...","['Litecoin', 'BTC', 'DOGE']",dlvr.it,False
902922,Richie Zhang,Singapore,Founder & key developer of https://t.co/IIsjK7...,2018-02-05 14:00:59,981.0,217.0,4329.0,False,2021-08-19 13:46:20,#BITCOIN - this is still a valid pattern. Don'...,"['BITCOIN', 'usd', 'forex', 'Daytrader', 'Trad...",Twitter Web App,False
1103895,@rylex4real,NaN,Cool headed guy,2021-07-06 20:31:32,13.0,290,287,False,2021-08-26 09:37:15,@CricketProtocol Join this project guys! It's ...,"['Cricketptotocol', 'Airdropcrypto', 'Airdropa...",Twitter for Android,False
927454,TalentintheCloud,"HQ London, Pan-Africa",Emerging Market FinTech executive search firm ...,2016-01-16 08:29:31,4147.0,4295.0,1954.0,False,2021-08-18 13:30:01,"7 keys insights in crypto over the last week, ...","['crypto', 'blockchain', 'bitcoin', 'defi']",Zoho Social,False


In [ ]:
df_tweets.dtypes


user_name           object
user_location       object
user_description    object
user_created        object
user_followers      object
user_friends        object
user_favourites     object
user_verified       object
date                object
text                object
hashtags            object
source              object
is_retweet          object
dtype: object

In [ ]:
df_tweets.shape

(1232809, 13)

In [ ]:
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1232809 entries, 0 to 1232808
Data columns (total 13 columns):
 #   Column            Non-Null Count    Dtype 
---  ------            --------------    ----- 
 0   user_name         1232784 non-null  object
 1   user_location     647838 non-null   object
 2   user_description  1088181 non-null  object
 3   user_created      1232807 non-null  object
 4   user_followers    1232805 non-null  object
 5   user_friends      1232805 non-null  object
 6   user_favourites   1232805 non-null  object
 7   user_verified     1232805 non-null  object
 8   date              1232805 non-null  object
 9   text              1232805 non-null  object
 10  hashtags          1216000 non-null  object
 11  source            1229435 non-null  object
 12  is_retweet        1232749 non-null  object
dtypes: object(13)
memory usage: 122.3+ MB


In [ ]:
#df_tweets[df_tweets['date']=="['ETH', 'BTC', 'Bitcoin']"]

In [ ]:
for x in tqdm(df_tweets['date']):
  try:
    pd.to_datetime(x)
  except:
    print(x)

  5%|▌         | 66739/1232809 [00:06<01:56, 10004.37it/s]

['ETH', 'BTC', 'Bitcoin']


 11%|█▏        | 139020/1232809 [00:14<01:53, 9662.97it/s]

['cryptocurrency', 'BSC', 'Bitcoin', 'Ethereum', 'ETH', 'Airdrop', 'bounty', 'ESHOP', 'Airdrops']


 15%|█▍        | 182327/1232809 [00:18<01:48, 9694.40it/s]

...

 57%|█████▋    | 698830/1232809 [01:11<00:58, 9067.66it/s]

['btc']


 87%|████████▋ | 1068935/1232809 [01:50<00:17, 9582.66it/s]

['YieldFarming', 'Airdrop', 'PancakeSwap', 'Giveaway', 'BTC', 'BNB', 'Binance', 'BSC', 'cryptocurrency']


100%|██████████| 1232809/1232809 [02:07<00:00, 9689.19it/s]


In [ ]:
def datetime_checker(x):
  for y in tqdm(x):
    try:
      return pd.to_datetime(x)
    except:
      return x

def is_datetime(x):
  for y in tqdm(x):
    try:
      pd.to_datetime(x)
      return True
    except:
      return False

df_tweets['datetime'] = df_tweets['date'].apply(datetime_checker)
df_tweets['is_datetime'] = df_tweets['date'].apply(is_datetime)

Streaming output truncated to the last 5000 lines.
  0%|          | 0/19 [00:00<?, ?it/s]

In [ ]:
df_tweets = df_tweets[df_tweets.is_datetime != False]

In [ ]:
#df_tweets.info()

In [ ]:
df_tweets['datetime'] = pd.to_datetime(df_tweets['datetime'])
df_tweets.info()

In [ ]:
df_tweets['datetime'].describe(datetime_is_numeric=True)

In [ ]:
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [ ]:
df_tweets['text'].replace({'https': '<br>'}, regex=True)

In [ ]:
# https://gist.github.com/sebleier/554280
# we are removing the words from the stop words list: 'no', 'nor', 'not'
stopwords= ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"]

In [ ]:
def preprocess_text(text_data, stopwords):

    # tqdm is for printing the status bar
    preprocessed_text = []

    for sentence in tqdm(text_data):

        if sentence is None:
          preprocessed_text.append("")
        else:
          sent = decontracted(sentence)
          sent = sent.replace('\\r', ' ')
          sent = sent.replace('\\n', ' ')
          sent = sent.replace('\\"', ' ')
          sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
          sent = re.sub(r'\W*\b\w{1,2}\b', '', sent)
          sent = re.sub('https?:\/\/\S+', '', sent)
          sent = ' '.join(e for e in sent.split() if e.lower() not in stopwords)
          preprocessed_text.append(sent.lower().strip())
    
    return preprocessed_text

In [ ]:
preprocessed_text = preprocess_text(df_tweets['text'].values, stopwords)

In [ ]:
len(preprocessed_text)

In [ ]:
preprocessed_text

In [ ]:
def cleanse_sentence(sentence):

  test = sentence.split()
  

  try:
    #test = test.remove(test[-1])
    val = test.index('https')
    words = [test[val], test[val+1], test[val+2]]
  except:
    return sentence.replace("https", "")
  
  test.remove(words[0])
  test.remove(words[1])
  test.remove(words[2])
  
  
  

  return " ".join(test)

In [ ]:
preprocessed_text = list(map(cleanse_sentence, preprocessed_text))

In [ ]:
preprocessed_text

In [ ]:
df_tweets['text'] = preprocessed_text
df_tweets['text'][0]





In [ ]:
 
df_first_half = df_tweets.sort_values('datetime',ascending=True).head(int(df_tweets.shape[0]*.5))
df_second_half = df_tweets.sort_values('datetime',ascending=False).head(int(df_tweets.shape[0]*.5))


In [ ]:
pip install --upgrade vaderSentiment

In [ ]:
#pip install --upgrade pyspark


In [ ]:
def getSubjectivity(twt):
  try:
    return TextBlob(twt).sentiment.subjectivity
  except:
    return twt
   

def getPolarity(twt):
  return TextBlob(twt).sentiment.polarity


df_first_half['Subjectivity'] = df_first_half['text'].apply(lambda x: getSubjectivity(x))
df_first_half['Polarity'] = df_first_half['text'].apply(lambda x: getPolarity(x))



In [ ]:
df_first_half.sample(50)

In [ ]:
from nltk.tokenize import TweetTokenizer
tokenizer = TweetTokenizer()
df_first_half['text_token'] = df_first_half['text'].apply(tokenizer.tokenize)



In [ ]:
df_first_half['text_token'].sample(10)


In [ ]:
# df_first_half['stemmed_text'].sample(50)

In [ ]:
lm = nltk.WordNetLemmatizer()
nltk.download('wordnet')

def lemmatizer_on_text(data):
    text = [lm.lemmatize(word) for word in data]
    return data
df_first_half['lemm_text'] = df_first_half['text_token'].apply(lambda x: lemmatizer_on_text(x))


In [ ]:
df_first_half['lemm_text'].head()

In [ ]:
#sentiment analysis using vader
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

scores = []
for sentence in df_first_half['text']:
  score = analyser.polarity_scores(sentence)
  scores.append(score)
  
scores = pd.DataFrame(scores)

df_first_half['Compound'] = scores['compound']
df_first_half['Negative'] = scores['neg']
df_first_half['Neutral'] = scores['neu']
df_first_half['Positive'] = scores['pos']


In [ ]:
df_first_half.sample()

In [ ]:
from collections import Counter
d = Counter(" ".join(df_first_half["text"]).split()).most_common(20)




In [ ]:
df_freq_words = pd.DataFrame(d, columns=['Word', 'Frequency'])

df_freq_words

In [ ]:
# Top 20 Most Frequent Words


b = b.nlargest(columns="Count", n = 10)
plt.figure(figsize=(30,20))
axis = sns.barplot(data=df_freq_words, x='Word', y='Frequency')
axis.set(ylabel='Frequency')
plt.title('Top 10 Most Frequent Words')
plt.show()

In [ ]:
def getAnalysis(score):
    if score < 0:
        return 'Negative'
    elif score > 0:
        return 'Positive'
    else:
        return 'Neutral'

df_first_half['Analysis_Polarity'] = df_first_half['Polarity'].apply(getAnalysis)




In [ ]:
#plot a cloud for negative tweets

data_neg = df_first_half.loc[df_first_half['Analysis_Polarity'] == 'Negative']
plt.figure(figsize = (20,20))
wc = WordCloud(max_words = 6000 , width = 1600 , height = 800,
               collocations=False).generate(" ".join(str(v) for v in data_neg['lemm_text']))
plt.imshow(wc)

In [ ]:
#plot a cloud for positive tweets
data_pos = df_first_half.loc[df_first_half['Analysis_Polarity'] == 'Positive']
plt.figure(figsize = (20,20))
wc = WordCloud(max_words = 8000 , width = 1600 , height = 800,
               collocations=False).generate(" ".join(str(v) for v in data_pos['lemm_text']))
plt.imshow(wc)

In [ ]:
df_first_half.describe()


In [ ]:
# print('Printing positive tweets:\n')
# j=1
# sortedDF = df_first_half.sort_values(by=['Polarity']) #Sort the tweets
# for i in range(0, sortedDF.shape[0] ):
#     if( sortedDF['Analysis_Polarity'][i] == 'Positive'):
#         print(str(j) + ') '+ sortedDF['text'][i])
#         print()
#         j= j+1